<a href="https://colab.research.google.com/github/BZoennchen/musical-interrogation/blob/main/partIV/melody-transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Die folgenden 5 Zellen sind für die Ausführung im Colab nötig.

In [ ]:
#@title clone git repository
%%capture
!rm -rf musical-interrogation
!git clone https://github.com/BZoennchen/musical-interrogation.git

In [ ]:
#@title move into directory
%%capture
import zipfile
import os
os.chdir('musical-interrogation/partIV')

In [ ]:
#@title install dependencies to play sound
%%capture
print('installing fluidsynth...')
!apt-get install fluidsynth > /dev/null
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
print('done!')

In [ ]:
#@title install dependencies to show score in music notation
%%capture
print('installing musescore3...')
!apt-get install musescore3 > /dev/null
print('done!')

In [ ]:
#@title install python libs
%%capture
!pip install torch torchview music21 matplotlib fluidsynth midi2audio

# Transformer

**AICA Crashkurs, Dr. Benedikt Zönnchen**

Auch wenn das Modell in diesem Notebook komplizierter scheint als unser LSTM, der Kern des Transformers -- der Attention-Mechanismus -- ist in der Klasse ``Head`` implementiert.
Alles drum herum dient der Optimierung (Vermeidung von Überanpassung und "aufblasen" der Netzwerkkomplexität).

In [ ]:
import zipfile
# Entpacke die zip-Datei, welche die Trainingsdaten enthält in den richtigen Ordner.
with zipfile.ZipFile('./../data/erk.zip', 'r') as zip_ref:
    zip_ref.extractall('./../deutschl/')

In [ ]:
import sys
import os
sys.path.append("..") 

import matplotlib.pyplot as plt
from torchview import draw_graph

import music21 as m21
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from datetime import datetime

from preprocess import load_songs_in_kern, GridEncoder, StringToIntEncoder
from preprocess import TERM_SYMBOL, TIME_STEP
from dataset import ScoreDataset

from utils import score_to_wav
from IPython.display import Audio

import graphviz
import numpy as np

graphviz.set_jupyter_format('png')

torch.manual_seed(0);

In [ ]:
# this takes a while!!!
# load ker files and transform them into m21.Scores
scores = load_songs_in_kern('./../deutschl/erk')

In [ ]:
Audio(score_to_wav(scores[0], 'score1.wav'))

In [ ]:
scores[0].show()

Als nächstes verwandeln wir die Noten in gut leserliche Zeichenketten, wobei jedes Event durch genau eine Zeichenkette repräsentiert wird.

Dies übernimmt der ``GridEncoder``. Dieser transponiert die Musikstücke zusätzlich nach C-Dur.
Dieser filtert zugleich Musikstücke heraus, welche wir mit unserem ``time_step`` nicht abbilden können.
Z.B. wenn ``time_step = 1/8`` dann können wir keine ``1/16``-Noten oder auch ``1/8 + 1/16``-Noten abbilden. 

In [ ]:
# this takes a while
time_step = 1/16
print(f'one timestep represents {time_step} beats')

encoder = GridEncoder(time_step)
enc_songs, invalid_song_indices = encoder.encode_songs(scores)

print(f'there are {len(enc_songs)} valid songs and {len(invalid_song_indices)} songs')

In [ ]:
scores[invalid_song_indices[0]].show()

Wir können ein Musikstück in der codierten Form ausgeben:

In [ ]:
' '.join(enc_songs[0])

In [ ]:
print(f'longest melody: {max(len(m) for m in enc_songs)}')
print(f'shortest melody: {min(len(m) for m in enc_songs)}')

Da der Computer besser mit Zahlen umgehen kann bauen wir uns eine Abbildung von den jeweiligen Zeichenketten zu Zahlen $$\{0, 1, 2, \ldots, m-1\}$$ und umgekehrt. Dies übernimmt ``StringToIntEncoder``:

In [ ]:
string_to_int = StringToIntEncoder(enc_songs)
print(f'number of unique symbols: {len(string_to_int)}')

In [ ]:
encoded_symbol = string_to_int.encode(enc_songs[0][0])
print(f'midi-ptich {enc_songs[0][0]} is encoded to number {encoded_symbol}')
print(f'number {encoded_symbol} is decoded to midi-pitch {string_to_int.decode(encoded_symbol)}')

## 2. Konstruktion der Trainingsdaten

``ScoreDataset`` verwaltet unsere Daten und lässt uns in Kombination über einen ``DataLoader`` bequem Sequenzen (d.h. Teile eines Stücks) der Länge ``sequence_len`` (Zeitschritte) laden

In [ ]:
sequence_len = 64 # this is a hyperparameter!
dataset = ScoreDataset(enc_songs=enc_songs, stoi_encoder=string_to_int, sequence_len=sequence_len, in_between=True)

``sequence_len * time_step`` ergibt die Zeit (bzw. ist im Fall einer 4/4 Signatur ``sequence_len * (time_step/0.25)`` die Anzahl der Beats die wir beim Lernen betrachten.

In [ ]:
print(f'while training we are looking at {sequence_len * (time_step/0.25)} beats')

Wir teilen die Daten nun in Trainings-, Validierungs-, und Testdaten auf.

+ Trainingsdaten: Verwenden wir zum Training unseres Modells / Melodiegenerators
+ Validierungsdaten: Verwenden wir um unseren Lernerfolg während des Trainings zu vergleichen
+ Testdaten: Verwenden wir am Ende des Trainings

In [ ]:
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [0.8, 0.1, 0.1])

## 3. Modelldefinition

In [ ]:
##### start hyperparameters #####
batch_size = 64
n_embd = 32 # has to be devisible by n_heads
n_heads = 2
n_blocks = 2
dropout = 0.2

criterion = torch.nn.CrossEntropyLoss()
vocab_size = len(string_to_int)

learning_rate = 0.001
n_epochs = 10
eval_interval = 100

if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    #torch.backends.mps.empty_cache()
else:
    device = torch.device('cpu')

##### end hyperparameters #####

#device = 'cpu'
print(f'{device=}')

In [ ]:
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size,shuffle=True)

Lassen Sie uns die Klasse ``Head`` besprechen, denn diese implementiert den sog. **Attention-Head** des Transformers, d.h. seinen Kern.
Der ``Head`` besteht aus ``nn.Linear``-Layern einem Buffer und einem ``nn.Dropout``-Layer.
``nn.Linear`` realisiert eine einfache lineare Transformation:

$$\mathbf{y} = \mathbf{x}^\top \mathbf{W} + \mathbf{b}$$

wobei in unserem Fall $\mathbf{b} = \mathbf{0}$ ist, da ``bias=False`` gilt.
Es bleibt also $\mathbf{y} = \mathbf{x}^\top \mathbf{W}$.

Wir haben drei solcher Transformationen und jede Transformiert $\mathbf{X}$ (alle Elemente einer Sequenz) von einem ``n_embd``-dimensionalen Raum in einen ``head_size``-dimensionalen Raum. D.h.

$$\mathbf{K} = \mathbf{X} \mathbf{K}$$

$$\mathbf{Q} = \mathbf{X} \mathbf{Q}$$

$$\mathbf{V} = \mathbf{X} \mathbf{V}$$

$\mathbf{K}$ sind die sog. **Keys**, $\mathbf{Q}$ die sog. **Querrys**, und $\mathbf{V}$ die sog. **Values**.
Beachten Sie, dass diese Matrizen Werte für alle Elemente einer Sequenz enthalten.
Z.B. enthält $\mathbf{K}$ alle Keys der Elemente einer Sequenz der länge ``sequence_len``.

Was wir berechnen wollen, ist die **Attention** die jedes Element in einer Sequenz zu jedem anderen Element spendet.
Dabei stellt jedes Element eine Anfrage (ein Querry) und sucht damit nach einem passenden Schlüssel.
Je besser Schlüssel und Querry zusammenpassen desto größer ist deren Produkt

$$\mathbf{W} = \mathbf{Q}\mathbf{K}^\top.$$

Da $\mathbf{Q}$ und $\mathbf{K}$ Matrizen sind, ergibt ihr Produkt eine Matrix.
Deren Einträge bestehen aus den ganzen Skalarprodukten der einzelnen Keys und Querrys.
Die Zeilen dieser Matrix $\mathbf{W}$ werden durch *softmax* zu Wahrscheinlichkeitsverteilungen.

Am Ende multiplizieren wir $\mathbf{W} \mathbf{V}$. Die $i$-te Zeile $\mathbf{w}_i$ in $\mathbf{W}$ gewichtet die Werte in $\mathbf{V}$ für das $i$-te Element.

$$\mathbf{w}_i \mathbf{V}$$

ist das gewichtete Mittel aller Sequenzwerte für das $i$-Element der Sequenz.

Da wir nicht in die Zukunft sehen können, maskieren wir das Gewicht für die **Attention** von $i$ auf $j$ sofern $j > i$. Diese Gewichte setzten wir auf 0.
Das wird druch die Zeile

```
wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
```

bewirkt.

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """
    
    def __init__(self, head_size, sequence_len, dropout):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(sequence_len, sequence_len)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) # B, T, head_size
        q = self.query(x) # B, T, head_size
        _, _, head_size = q.shape #???
        
        wei = q @ k.transpose(-2, -1) * (head_size ** (-0.5)) # B, T, head_size @ B, head_size, T => B, T, T
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        v = self.value(x) # B, T, head_size
        out = wei @ v # T, T @ B, T, head_size => B, T, head_size
        return out
        

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size, sequence_len, dropout):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, sequence_len, dropout) for _ in range(n_heads)])
        self.proj = nn.Linear(n_heads * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [ ]:
class FeedForward(nn.Module):
    
    def __init__(self, n_embd, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), 
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
        
    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    
    def __init__(self, n_embd, n_heads, sequence_len, dropout):
        super().__init__()
        # this could be different
        head_size = n_embd // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size, sequence_len, dropout)
        self.ffwd = FeedForward(n_embd, dropout)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # residual connection
        x = x + self.ffwd(self.ln2(x)) # residual connection
        return x

In [ ]:
class TransformerDecoder(nn.Module):
    
    def __init__(self, vocab_size, sequence_len, n_embd, n_heads, n_blocks, dropout):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(sequence_len, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_heads, sequence_len, dropout) for _ in range(n_blocks)])
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx):
        B, T = idx.shape
        
        token_emb = self.token_embedding_table(idx) # B, T, n_embd
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # T, n_embd        
        x = token_emb + pos_emb # B, T, n_embd + T, n_embd => B, T, n_embd
        x = self.blocks(x) # B, T, head_size
        logits = self.lm_head(x) # B, T, vocab_size
        return logits
    
    def generate(self, idx, max_new_tokens):
        # idx = B, T
        count = 0
        with torch.no_grad():
            while count < max_new_tokens:
                idx_crop = idx[:, -block_size:]
                logits, loss = self(idx_crop) # B, T, C
                probs = F.softmax(logits[:,-1,:], dim=1) # B, C
                idx_next = torch.multinomial(probs, num_samples=1)
                if idx_next == stoi_encoder.encode(TERM_SYMBOL):
                    break
                idx = torch.cat((idx, idx_next), dim=1)
                count += 1
            
            return idx


In [ ]:
model = TransformerDecoder(vocab_size, sequence_len, n_embd, n_heads, n_blocks, dropout)
model.to(device);

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].shape)

Die folgende Zelle dient lediglich der Visualisierung unseres Modells und hat keine Auswirkung auf die Berechnung.
Die Komplexität ist recht hoch da unser Modell aus mehrere Blöcke mit mehreren ``Head``s besteht. Das Training ist dementsprechend aufwendig.

In [ ]:
# (batch_size, sequence_len)
X_vis, y_vis = train_set[0:batch_size]
print(f'shape of X_vis: {X_vis.shape}')
print(f'shape of y_vis: {y_vis.shape}')
print(f'number of different symbols {vocab_size}')
X_vis, y_vis = X_vis.to(device), y_vis.to(device)
model_vis = TransformerDecoder(vocab_size, sequence_len, n_embd, n_heads, n_blocks, dropout)
model_graph = draw_graph(model_vis, input_data=X_vis, device=device)
model_graph.visual_graph

## 5. Training

Zum Training verwenden wir hier einen sog. ``DataLoader``. Dieser hilft uns dabei auf unsere Daten einfacher zugreifen zu können. Z.B., lassen wir unsere Daten vor dem Training durchmischen.

In [ ]:
def train_one_epoch(epoch_index, tb_writer, n_epochs):
    running_loss = 0.0
    last_loss = 0.0
    all_steps = n_epochs * len(train_loader)
    
    for i, data in enumerate(train_loader):
        local_X, local_y = data
        local_X, local_y = local_X.to(device), local_y.to(device)
        optimizer.zero_grad()
        outputs = model(local_X)
        
        #print(local_X.shape, local_y.shape)
        
        B, T, C = outputs.shape
        outputs = outputs.view(B*T, C)
        local_y = local_y.view(B*T)
        loss = criterion(outputs, local_y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % eval_interval == eval_interval-1:
            last_loss = running_loss / eval_interval  # loss per batch
            
            steps = epoch_index * len(train_loader) + (i+1)
            
            print(
                f'Epoch [{epoch_index+1}/{n_epochs}], Step [{steps}/{all_steps}], Loss: {last_loss:.4f}')
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
            
    return last_loss


In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
def train(n_epochs, respect_val=False, val_losses=[], train_losses=[]):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
    best_vloss = 1_000_000

    for epoch in range(n_epochs):    
        model.train(True)
        
        avg_loss = train_one_epoch(epoch, writer, n_epochs)
        train_losses.append(avg_loss)
        
        model.train(False)
        with torch.no_grad():
            running_vloss = 0.0
            
            for i, vdata in enumerate(val_loader):
                
                local_X, local_y = vdata
                local_X, local_y = local_X.to(device), local_y.to(device)
                            
                voutputs = model(local_X)
                
                B, T, C = voutputs.shape
                voutputs = voutputs.view(B*T, C)
                local_y = local_y.view(B*T)
                
                vloss = criterion(voutputs, local_y)
                running_vloss += vloss
                
            avg_vloss = running_vloss / (i+1)
            val_losses.append(vloss)
            
            print(
                f'Epoch [{epoch+1}/{n_epochs}], Train-Loss: {avg_loss:.4f}, Val-Loss: {avg_vloss:.4f}')
            
            writer.add_scalars('Training vs. Validation Loss', {'Training': avg_loss, 'Validation': avg_vloss}, epoch)
            writer.flush()
            
            if not respect_val or (respect_val and avg_vloss < best_vloss):
                best_vloss = avg_vloss
                model_path = './models/_model_{}_{}'.format(timestamp, epoch)
                print(f'save new model: {model_path}')
                torch.save(model.state_dict(), model_path)

In [ ]:
val_losses = []
train_losses = []
train(10, respect_val=True, val_losses=val_losses, train_losses=train_losses)
val_losses = list(map(lambda x : x.item(), val_losses))
train_losses = list(map(lambda x : x.item(), train_losses))

In [ ]:
plt.figure()
plt.plot(np.arange(1, len(val_losses)+1, 1), val_losses, label="val")
plt.plot(np.arange(1, len(train_losses)+1, 1), train_losses, label="train")

In [ ]:
print(f'there are the following models to choose from:')

for model_file in os.listdir('./models/'):
    print(f'./models/{model_file}')

In [ ]:
# loads a saved model
model_path = './models/pretrained_32_2_2'

if device.type == 'cpu':
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
else:
    model.load_state_dict(torch.load(model_path))
model.eval()

## 4. Melodiegenerierung

Gegeben einer Sequenz beliebiger Länge, dient die Funktion ``generate`` der Generierung eines neues neuen Musikstücks.
``temperature`` bestimmt wie stark die vom Modell gelernte Wahrscheinlichkeitsverteilung beachtet wird.

+ ``temperature`` gleich 1.0 bedeutet, dass von der Wahrscheinlichkeitsverteilung gesampelt wird.
+ ``temperature`` gegen unendlich bedeutet, dass gleichverteilt gesampelt wird (mehr Variation)
+ ``temperature`` gegen 0 bedeutet, dass die hohe Wahrscheinlichkeiten verstärkt werden (weniger Variation)

Sie können eine maximale Länge des Stücks festlegen und auch einen Anfang eines Stücks mitliefern.

In [ ]:
def next_event_number(idx, temperature:float):
    with torch.no_grad():
        outputs = model(idx[:,-sequence_len:])
        B, T, C = outputs.shape
        logits = outputs[:, -1, :]
        probs = F.softmax(logits / temperature, dim=1)  # B, C
        idx_next = torch.multinomial(probs, num_samples=1)
        return idx_next

In [ ]:
def generate(seq: list[str]=None, max_len:int=None, temperature:float=1.0):
    with torch.no_grad():
        generated_encoded_song = []
        start_sequence = [string_to_int.encode(TERM_SYMBOL)]*sequence_len
        if seq != None:
            start_sequence = start_sequence + [string_to_int.encode(char) for char in seq]
            idx = torch.tensor([start_sequence], device=device)
            generated_encoded_song = seq.copy()
        else:
            idx = torch.tensor([start_sequence], device=device)
        
        while max_len == None or max_len > len(generated_encoded_song):
            idx_next = next_event_number(idx, temperature)
            char = string_to_int.decode(idx_next.item())
            if idx_next == string_to_int.encode(TERM_SYMBOL):
                break
            idx = torch.cat((idx, idx_next), dim=1) # B, T+1, C
            generated_encoded_song.append(char)
            
        return generated_encoded_song

In [ ]:
# number of songs we want to generate
n_scores = 5
temperature = 0.6
after_new_songs = []
for _ in range(n_scores):
    encoded_song = generate(max_len=120,temperature=temperature)
    print(f'generated {" ".join(encoded_song)} conisting of {len(encoded_song)} notes')
    after_new_songs.append(encoded_song)

In [ ]:
after_generated_scores = encoder.decode_songs(after_new_songs)

In [ ]:
after_generated_scores[0].show()

In [ ]:
Audio(score_to_wav(after_generated_scores[0], 'a_g_song.wav'))

Wir können auch einen Teil bestehendes Musikstücks verwenden und diesen erweitern:

In [ ]:
' '.join(enc_songs[0])

In [ ]:
n_notes = 10
part = encoder.take_notes(enc_songs[0], n_notes)
' '.join(part)

In [ ]:
Audio(score_to_wav(encoder.decode_song(part), 'part.wav'))

In [ ]:
enc_song = generate(part, max_len=120,temperature=temperature)
' '.join(enc_song)

In [ ]:
song = encoder.decode_song(enc_song)

In [ ]:
song.show()

In [ ]:
Audio(score_to_wav(song, 'g_song.wav'))

## Fragen

+ Welche Unterschiede zwischen LSTM und Transformer kennen Sie?